# Review summarising LLM concept

In [1]:
prompt_few_shots = """
Take the role of a product advisor.
your task is to summarize the following product reviews delimited by a '/' symbol in a blog style short article, highlighting the main points.
Then, provide a list of pros and cons and a short overall conclusion.

### Example 1:
# Reviews:
"I recently purchased this tablet for casual use, and overall, I’m happy with my decision.
The screen is sharp and vibrant, which makes watching movies and browsing the web a joy.
The tablet is lightweight, so it’s comfortable to hold for long periods.
However, the battery life is just okay—it lasts about 6-7 hours with moderate usage,
which is fine for me but might not be enough for heavier users.
The app performance is smooth for basic tasks, though it does lag a bit when multitasking.
For the price, it’s a solid option." / "Disappointed with this tablet.
While the screen is decent, the overall performance is sluggish.
Apps take a while to load, and there’s a noticeable delay when switching between tasks.
The battery drains surprisingly fast, even when it’s just on standby.
On the plus side, it’s very portable and lightweight, but that’s about the only good thing I can say.
If you’re looking for a serious productivity device, look elsewhere."

# Blog style summary :
"A Mixed Bag for Everyday Use
When it comes to this tablet, users have mixed experiences.
On the positive side, the tablet offers a vibrant and sharp display, making it great for media consumption,
and its lightweight design is perfect for extended use.
However, performance can be a letdown for those who multitask, with some lag during app switching.
Battery life also divides opinions, being adequate for light users but underwhelming for more intensive usage."

# Pros:
  - Sharp, vibrant screen.
  - Lightweight and portable.
# Cons:
  - Sluggish performance with multitasking.
  - Mediocre battery life.

# Conclusion:
"This tablet is a reasonable choice for casual users
but might frustrate anyone looking for speed or long-lasting power"

### Example 2:
# Review:
“This e-reader is a game-changer for book lovers!
The e-ink display is crisp and feels just like reading on paper, even in direct sunlight.
The battery life is incredible—I’ve been using it daily for two weeks, and it’s still going strong.
I also appreciate the adjustable warm light for nighttime reading.
It’s easy to navigate and has enough storage for my entire library.
Highly recommend it for avid readers.” / “The e-reader works well enough, but I have mixed feelings.
The display is great for reading, but the device feels a bit slow when turning pages or navigating menus.
It also lacks some features I’d like, such as audiobook support.
The build quality feels a little cheap, but it does the job for basic reading. Not bad, but not amazing either.”

# Blog style summary:
"A Reliable Companion for Bookworms
The e-reader earns praise for its excellent e-ink display that mimics real paper and performs well in any lighting condition.
With adjustable warm light and weeks-long battery life, it’s a fantastic choice for reading enthusiasts.
However, not everyone is thrilled, with some noting slow menu navigation and the absence of additional features like audiobook support."

# Pros:
  - Crisp, paper-like display.
  - Exceptional battery life.
# Cons:
  - Sluggish menu navigation.
  - Lacks advanced features like audiobook support.

# Conclusion:
"This e-reader is an excellent choice for avid readers focused on the core reading experience,
though tech-savvy users might find it a bit limiting."

# Review:
{review_text}
"""

In [2]:
'''(imports are not in the first lines because
they needed to be in the same cell as the model)'''

# Imports
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import torch
!pip install -U bitsandbytes

# Load Mistral model for text generation
model_name = "mistralai/Mistral-7B-v0.3"

# AutoTokenizer instead
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Load the model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    load_in_4bit=True,
    device_map="auto"
)

# Initialize the text generation pipeline
generator = pipeline("text-generation", model=model, tokenizer=tokenizer)

# Set pad_token to eos_token
tokenizer.pad_token = tokenizer.eos_token

# Define device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Define the inference function
def run_inference_on_model(prompt: str, max_tokens: int = 200) -> str:
    """Run inference on the model

    Args:
        prompt (str): prompt to generate text from
        max_tokens (int, optional): maximum tokens to generate. Defaults to 100.

    Returns:
        str: generated text
    """

    inputs = tokenizer(prompt, return_tensors="pt", padding=True, truncation=True).to(device)

    outputs = model.generate(
        inputs.input_ids,
        max_new_tokens=max_tokens,
        pad_token_id=tokenizer.eos_token_id,
        eos_token_id=tokenizer.eos_token_id
    ).to(device)

    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Review text for testing
review_text = "This blender is absolutely fantastic! It blends everything smoothly, whether it's ice, vegetables, or fruits. I love the different speed settings, which give a lot of flexibility for various recipes. However, it’s quite loud, especially on the higher settings, so it can be a bit disruptive in the kitchen. The jar is also a bit heavy, which makes it a little difficult to handle, especially when it's full. Cleaning is easy, but the blade is really sharp, so I need to be careful. Overall, it's a solid blender for anyone who wants power and versatility in their kitchen."

# Format the prompt with review text
prompt = prompt_few_shots.format(review_text=review_text)

# Run inference
generated_text = run_inference_on_model(prompt, max_tokens=300)

# Print generated text
print(generated_text)

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.55G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

Device set to use cuda:0
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
/usr/local/lib/python3.10/dist-packages/bitsandbytes/nn/modules.py:451: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(



Take the role of a product advisor.
your task is to summarize the following product reviews delimited by a '/' symbol in a blog style short article, highlighting the main points.
Then, provide a list of pros and cons and a short overall conclusion.

### Example 1:
# Reviews: 
"I recently purchased this tablet for casual use, and overall, I’m happy with my decision.
The screen is sharp and vibrant, which makes watching movies and browsing the web a joy.
The tablet is lightweight, so it’s comfortable to hold for long periods.
However, the battery life is just okay—it lasts about 6-7 hours with moderate usage, 
which is fine for me but might not be enough for heavier users.
The app performance is smooth for basic tasks, though it does lag a bit when multitasking.
For the price, it’s a solid option." / "Disappointed with this tablet.
While the screen is decent, the overall performance is sluggish.
Apps take a while to load, and there’s a noticeable delay when switching between tasks.
The 